# Models

In [39]:
#IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.10f}'.format

In [40]:
#LOAD LOANSTATS
directory = '../../../data/'
ls = pd.read_hdf(directory + 'LS_CLEAN.h5', 'LS_CLEAN')
ls.sort_index(axis=1, inplace=True)

<br><br>
## 1. Data Preprocessing

### 1A. Train-Test Split

We split the `ls` dataset into a train and test part. We do this in a stratified fashion ensuring that the outcome classes (fully paid loans and not fully paid loans) are equally represented in each set. For the splitting algorithm, we use `sklearn`'s `train_test_split` function. This function creates random train and test subsets of the dataset. The flag `stratify` ensures that both classes are equally represented in each set. 

In [41]:
from sklearn.model_selection import train_test_split
ls_train, ls_test = train_test_split(ls, test_size=0.2, stratify=ls['OUT_Class'])

### 1B. Standard Scaling

Most of the models used later will assume all features are on similar scales. To achieve this, we use standardization to transform the numeric variables such that they have a mean of 0 and standard deviation of 1. We use `sklearn`'s `StandardScaler` function.

In [42]:
#IDENTIFY THE OUTCOME, DUMMY AND NUMERIC VARIABLES
var_list = set(ls.columns)
outcome_var_list = set(out_var for out_var in var_list if "OUT_" in out_var)
dummy_var_list = set(dummy for dummy in var_list if "D_" in dummy)
numeric_var_list = var_list - outcome_var_list - dummy_var_list

In [43]:
#STANDARD SCALER
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [44]:
#SCALE THE TRAINING SETS (fit_transform)
train_vars_scaled = pd.DataFrame(scaler.fit_transform(ls_train[list(numeric_var_list)]),
                                 index=ls_train.index, 
                                 columns=ls_train[list(numeric_var_list)].columns)

feature_train = pd.concat([train_vars_scaled, 
                           ls_train[list(dummy_var_list)]], 
                          axis=1).sort_index(axis=1)

outcome_train = ls_train[list(outcome_var_list)]

In [45]:
#STANDARDIZE THE TEST SET (transform)
test_vars_scaled = pd.DataFrame(scaler.transform(ls_test[list(numeric_var_list)]),
                                index=ls_test.index, 
                                columns=ls_test[list(numeric_var_list)].columns)

feature_test = pd.concat([test_vars_scaled, 
                          ls_test[list(dummy_var_list)]], 
                         axis=1).sort_index(axis=1)

outcome_test = ls_test[list(outcome_var_list)]


## Classification

In [47]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0)
model_logreg = logreg.fit(feature_train, outcome_test.iloc[:,0])


/Users/michal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [336144, 84037]

## Regression

## Trees and Forest

## K-Nearest Neighbors